# Data Profiling

### Import modules

In [1]:
import pandas
import numpy
import random
import datetime

### Helper function to generate data

In [2]:
def generate_dynamic_random_code(code_length):
    
    characters_string = "abcdefghijklmnopqrstuvwxyz1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    
    listchar = list(characters_string)
    
    placeholder_string = "-" * code_length
    
    placeholders = list(placeholder_string)
    
    for index in range(len(placeholders)):
    
        random_index = random.randint(0,len(listchar) - 1)
    
        character = listchar[random_index]
    
        placeholders[index] = character
    
    code = "".join(placeholders)
    
    return code

In [3]:
def generate_float_frame(row_count, column_count, assign_col_names = False, nullperc = 0.20):
    
    data = pandas.DataFrame(numpy.random.random((row_count, column_count)))
    
    if assign_col_names == True:
        
        columns = []
        while len(columns) < column_count:
            
            columns.append(generate_dynamic_random_code(8))
            
        data.columns = columns
    
    if nullperc < 0 or nullperc >100:
        nullperc = 0.2
        print("percentage of nulls defaulted to 0.2")
    
    total_cells = data.shape[0] * data.shape[1]
    
    null_cells = int(total_cells * nullperc)
    
    for iteration in range(null_cells):
        
        r_index = random.randint(0,row_count - 1)
        c_index = random.randint(0, column_count - 1)
        data.iloc[r_index, c_index] = None
    
    return data

### Import/Create Data

In [4]:
data = generate_float_frame(row_count = 1230,
                           column_count = 12,
                           assign_col_names = True)

In [5]:
data.head(100)

,oz4sImNM,QklUdtpe,9B1jajmb,b3UnTXhT,9PbOe6cN,5J5STovq,iggOYB2i,IEKgVLGf,fnq53eii,r6OTifRf,WLCNfCHn,EZotBcml
0,0.361646,0.131098,0.744973,0.120421,0.970402,NaN,0.027533,0.904894,0.926974,0.519154,0.348677,0.008613
1,0.239089,0.213063,NaN,0.028472,0.991029,0.072130,0.946788,0.607698,0.092025,0.974765,0.305458,NaN
2,0.231864,0.659968,0.015289,0.203412,0.746992,0.516286,NaN,0.407845,0.146956,0.902209,0.772252,0.763871
3,0.849810,0.641033,0.844916,NaN,0.898727,0.479799,0.509571,NaN,0.594402,0.183582,0.311205,0.432532
4,0.267479,NaN,0.878679,0.668174,0.431083,0.924897,NaN,NaN,0.399594,0.360563,0.193765,0.658069
5,0.462159,0.373497,0.954573,0.015037,NaN,0.669786,0.822526,NaN,0.102936,0.306944,0.467404,NaN
6,0.871761,0.719845,0.603804,0.977027,NaN,NaN,0.723907,NaN,0.728667,0.501857,0.059652,0.532189
7,NaN,0.463466,0.251771,NaN,0.489177,0.753860,0.986916,0.705699,0.160152,0.609009,0.803649,0.393253
8,NaN,0.212126,0.799849,0.170790,NaN,0.346856,0.656321,0.466712,0.619067,0.139851,0.157324,0.359978
9,0.162670,0.192916,0.319804,0.057818,0.273313,0.462813,0.843842,0.689299,0.905637,0.700267,0.391609,0.636306


## Extract Profile

### Get Statistical Summary

In [6]:
describe = data.describe(include = "all")
describe_transposed = describe.T
describe_final = describe_transposed.reset_index()

In [7]:
describe_final

,index,count,mean,std,min,25%,50%,75%,max
0,oz4sImNM,1010.0,0.505099,0.290326,0.000064,0.251582,0.489642,0.759540,0.999618
1,QklUdtpe,1007.0,0.507598,0.285220,0.000037,0.263774,0.508248,0.752368,0.999941
2,9B1jajmb,1009.0,0.502227,0.294303,0.000510,0.247018,0.511662,0.751042,0.999634
3,b3UnTXhT,1026.0,0.491163,0.294576,0.003785,0.238766,0.487164,0.746385,0.999515
4,9PbOe6cN,1003.0,0.504494,0.280803,0.000511,0.269173,0.496569,0.748831,0.999833
5,5J5STovq,1005.0,0.491256,0.283416,0.000457,0.265124,0.480558,0.738775,0.999028
6,iggOYB2i,1012.0,0.498287,0.289934,0.000407,0.253116,0.488216,0.743271,0.999550
7,IEKgVLGf,1020.0,0.521591,0.287551,0.000349,0.286908,0.521135,0.770565,0.999105
8,fnq53eii,990.0,0.495751,0.293313,0.003547,0.233799,0.482160,0.745319,0.998822
9,r6OTifRf,1009.0,0.494888,0.288787,0.000547,0.254095,0.493092,0.735458,0.997248


### Get Data Types

In [8]:
dtypes = data.dtypes

In [9]:
dtypes_dataframe = pandas.DataFrame(dtypes, columns = ["data_type"])

In [10]:
dtypes_final = dtypes_dataframe.reset_index()

In [11]:
dtypes_final

,index,data_type
0,oz4sImNM,float64
1,QklUdtpe,float64
2,9B1jajmb,float64
3,b3UnTXhT,float64
4,9PbOe6cN,float64
5,5J5STovq,float64
6,iggOYB2i,float64
7,IEKgVLGf,float64
8,fnq53eii,float64
9,r6OTifRf,float64


### Get Null composition

In [12]:
filled_count_series = data.notnull().sum().astype(int)
null_count_series = data.isnull().sum().astype(int)
totalcount = data.shape[0]

In [13]:
percent_null =  null_count_series / totalcount
percent_filled = filled_count_series / totalcount

In [14]:
percent_null = percent_null.apply('{0:.2%}'.format)
percent_filled = percent_filled.apply('{0:.2%}'.format)

In [15]:
percent_null = percent_null.reset_index().rename(columns = {0:"null_percent"})
percent_filled = percent_filled.reset_index().rename(columns = {0:"non_null_percent"})
filled_count_series = filled_count_series.reset_index().rename(columns = {0:"non_null_counts"})
null_count_series = null_count_series.reset_index().rename(columns = {0:"null_counts"})

In [16]:
from functools import reduce

In [17]:
null_info_dataframe = reduce(lambda left,right: pandas.merge(left,right,on='index'), [percent_null,
                                                                                 percent_filled,
                                                                                 filled_count_series,
                                                                                 null_count_series])

In [18]:
null_info_dataframe

,index,null_percent,non_null_percent,non_null_counts,null_counts
0,oz4sImNM,17.89%,82.11%,1010,220
1,QklUdtpe,18.13%,81.87%,1007,223
2,9B1jajmb,17.97%,82.03%,1009,221
3,b3UnTXhT,16.59%,83.41%,1026,204
4,9PbOe6cN,18.46%,81.54%,1003,227
5,5J5STovq,18.29%,81.71%,1005,225
6,iggOYB2i,17.72%,82.28%,1012,218
7,IEKgVLGf,17.07%,82.93%,1020,210
8,fnq53eii,19.51%,80.49%,990,240
9,r6OTifRf,17.97%,82.03%,1009,221



### Merge all

In [19]:
merged = reduce(lambda left,right: pandas.merge(left,right,on='index', how = "left"), [null_info_dataframe,
                                                                        dtypes_final,
                                                                        describe_final])

### Cleanup

#### drop count column

In [20]:
merged["row_count"] = merged["non_null_counts"] + merged["null_counts"]

merged.drop(["count"], axis = 1, inplace = True)

#### change types

In [21]:
merged["data_type"] = merged["data_type"].astype(str)

#### round values

In [22]:
rounded_places = 4

for column in ["mean", "std", "min", "25%", "50%", "75%", "max"]:
    
    merged[column] = merged[column].round(rounded_places)

### Create dictionary and print

In [23]:
profile_dict = {}

In [24]:
merged_attributes = list(merged.columns)[1:]

for index in range(merged.shape[0]):
    
    column = merged.loc[index, "index"]
    column_padded = "  " + column + "  "
    print("\n\n", column_padded.center(44, "-"),"\n",sep = "")    
    profile_dict[column] = []    
    for attr in merged_attributes:
        attr_ = attr + ":"
        value = merged.loc[index, attr]
        value_string = str(value)
        
        print(attr_.ljust(22, " "), value_string.rjust(22, " "), sep = "")    
        
        if type(value) == numpy.int64:
            profile_dict[column].append({attr:int(value)})
        else:
            profile_dict[column].append({attr:value})



----------------  oz4sImNM  ----------------

null_percent:                         17.89%
non_null_percent:                     82.11%
non_null_counts:                        1010
null_counts:                             220
data_type:                           float64
mean:                                 0.5051
std:                                  0.2903
min:                                  0.0001
25%:                                  0.2516
50%:                                  0.4896
75%:                                  0.7595
max:                                  0.9996
row_count:                              1230


----------------  QklUdtpe  ----------------

null_percent:                         18.13%
non_null_percent:                     81.87%
non_null_counts:                        1007
null_counts:                             223
data_type:                           float64
mean:                                 0.5076
std:                                  0.2852
min:

### Export results as .json and .csv

#### Set export pathway

In [25]:
exportpath = "/users/danielcorcoran/desktop/github_repos/python_nb_data_profiling/"

#### Export as json

In [26]:
with open(exportpath + "profile.json", "w") as fileobj:
    json.dump(profile_dict, fileobj)
    
fileobj.close()

#### Export as csv

In [27]:
merged.to_csv(exportpath + "profile.csv", index = False)